# Monasca Bootcamp
## Hands on Lab

https://github.com/witekest/monasca-bootcamp

---
### Witek Bedyk

company: Fujitsu EST

email: <witold.bedyk@est.fujitsu.com>

### Stefano Canepa

company: Hewlett Packard Enterprise

email: <stefano.canepa@hpe.com>


# The Monasca team attending the Sydney Summit

* James Gu (SUSE)

* and more

# Introduction

In this workshop we'll guide you into a journey into Monasca features. We'll use a Jupyter notebook as a guide to not get lost in all the microservices that build Monasca solution. A Jupyter notebook is runnig on each VM and you can check it out from github and try on your own in congiancion with monasca-docker.

The required softwares to replay this workshop on you computer are:
* Jupyter (http://jupyter.org/)

* python-monascaclient (https://github.com/openstack/python-monascaclient)

* monasca-docker (https://github.com/monasca/monasca-docker)

* monasca-agent (https://github.com/openstack/monasca-agent)

# Running the notebook on your workshop instance
---
* Set up the SSH tunnel to your instance
    
    * `ssh -i monasca_workshop.pem -NfL localhost:8888:localhost:8889 ubuntu@<your_instance_ip>`
    
* SSH to your instance and start the notebook

    * `ssh -i monasca_workshop.pem ubuntu@<your_instance_ip>`
    
    * `cd monasca-bootcamp`
    
    * `git pull`
    
    * `/opt/jupyter/bin/jupyter notebook --no-browser --port=8889 --notebook-dir .`
    
* Copy the URL and open it in your local browser. **Remember to set the port number to `8888`**.

* Open `MonascaBootcamp.ipynb` notebook.

## For Windows users

* You can use your favourite SSH client

* We recommend Git BASH, BASH emulator with SSH client. (included on USB media)

* Cmder ssh client works out of the box (http://cmder.net/)

# Agenda
---
* Architecture and Overview
* API, CLI (python-monascaclient) and client hands-on
* Agent hands-on
* Horizon, Grafana, Kibana hands-on

# Architecture
---
![Monasca Architecture](architecture.png)

# Logging
---
![Logging Architecture](Logging_Architecture.png)

# Download and deploy Monasca services

In [ ]:
%cd
!git clone https://github.com/monasca/monasca-docker.git
%pushd monasca-docker
!docker-compose up -d
%popd

# Check services

In [ ]:
%pushd monasca-docker
!docker-compose ps
%popd

# Import libraries and initialization
---
Let's first import some libraries used in the rest of the notebook.

In [ ]:
import datetime
import json
import os
import time

# Import the Monasca client
from monascaclient import client

# Initialize the Keystone and Monasca Client
---
We'll be connecting to the DevStack VM and using the `mini-mon` project and username.

In [ ]:
KEYSTONE_URL = 'http://keystone:5000/v3'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'
MON_API_URL = 'http://monasca:8070/v2.0'

In [ ]:
# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME
%env OS_IDENTITY_API_VERSION=3
%env MONASCA_API_URL=$MON_API_URL

# Create the Monasca client
monasca_client = client.Client(api_version='2_0',
                      username=os.environ.get('OS_USERNAME'),
                      password=os.environ.get('OS_PASSWORD'),
                      auth_url=os.environ.get('OS_AUTH_URL'),
                      project_name=os.environ.get('OS_PROJECT_NAME'),
                      user_domain_name="Default",
                      project_domain_name="Default",
                      auth_version=os.environ.get('OS_IDENTITY_API_VERSION'),
                      endpoint='http://localhost:8070/v2.0')

# Using the API
---

The [Monasca API](https://github.com/openstack/monasca-api/blob/master/docs/monasca-api-spec.md) has the following resources:

* Versions

* Metrics

* Metrics Measurements

* Metrics Statistics

* Metrics Names

* Notification Methods

* Alarm Definitions

* Alarms

* Alarms Count

* Alarms State History

Please consult the [spec](https://github.com/openstack/monasca-api/blob/master/docs/monasca-api-spec.md) for the full details.

# Metrics
---
* POST /v2.0/metrics

* name (string(255), required) - The name of the metric. Naming conventions for metric names:

    * lowercase, `.` to delimit groups, `_` (snake case) to delimit words, with a unit of measurement often as the suffix.
    
    * e.g. cpu.user_perc, kafka.consumer_lag

* dimensions ({string(255): string(255)}, optional) - A dictionary consisting of (key, value) pairs used to uniquely identify a metric and slice and dice on.

* timestamp (string, required) - The timestamp in milliseconds from the Epoch.

* value (float, required) - Value of the metric.

* value_meta ({string(255): string(2048)}, optional) - A dictionary consisting of (key, value) pairs used to describe the metric.

    * Examples: status_code, msg

* tenant_id: Tenant ID to create metrics on behalf of.

    * This parameter can be used to submit metrics from one tenant, to another.
    * Requires the delegate role.

# Dimensions
---

* A dictionary of (key, value) pairs that are used to uniquely identify a metric.

* Used to slice and dice metrics when querying.

* Examples: hostname, service, component, region zone, resource_id, ...

* Dimensions can be anything you want, but naming conventions should be adopted for consistency.

* Examples of dimension keys are the following: hostname, region, zone, service, component, process, ...

# Example: Metrics request body
---
In this example, we are reporting the status of a HTTP check as a binary value, 0 or 1, for a specific host. The API is down, and the status code and msg returned are returned as meta data.
```
{
	name: http_status,
	dimensions: {
		url: http://service.domain.com:80,
		region: uswest,
		zone: 1,
		service: compute
	}
	timestamp: 1461600900000, /* milliseconds */
	value: 1.0,
	value_meta: {
		status_code: 500,
		msg: Internal server error
	}
}
```

# Help: Create metrics
---

In [ ]:
!monasca help metric-create

# Example: Create a metric
Let's create a metric with the metric `openstack.handson_status` and some dimensions 

In [ ]:
!monasca metric-create --dimensions region=NSW,county=Cumberland,city=Sydney,session=monasca openstack.handson_status 1.0

# List metrics
Lists all the unique metrics in the system

* GET /v2.0/metrics

* A unique metric is identitifed by it's name and dimensions.

* Starttime, endtime, offset and limit paramaters are available.

* Note, if you are monitoring OpenStack resources, such as VMs, based on the amount of churn (VMs being created/destroyed) in the system and your retention policy, it is recommended to specify the starttime, to limit the amount of metrics returned, such that queries return in a reasonable amount of time.

# Help: List Metrics
---


In [ ]:
!monasca help metric-list

# Example: List metrics

In [ ]:
!monasca metric-list --limit 10

# Exercise 1: List metrics and filter on name, dimensions
---
Extend the following command to filter the list and find the previously generated metric.

In [ ]:
!monasca metric-list # put your options here

# Example: Create a function to get metrics using the Monasca client
---
Now let's try to create a function which returns the list of metrics using the Monasca Python client.
We will use the previously instantiated 'monasca_client' object and its 'metrics.list' method.
All the CLI options can be used as named arguments.


In [ ]:
import json

def get_metrics(names=[None], dimensions={}, limit=10):
    metrics = []
    for name in names:        
        # Invoke the Monasca client
        metrics = metrics + monasca_client.metrics.list(name=name, dimensions=dimensions, limit=limit)
    return metrics

metrics = get_metrics(['openstack.handson_status'])
print json.dumps(metrics, indent=4)

# Metric names
* List all the metric names stored in the service

* GET /v2.0/metrics/names

* Query parameters: tenant_id, offset, limit and dimensions

* Very useful for user-interfaces that need to display the names of all the metrics in the system.

    * Querying all metrics and determining all the distinct metric names is too costly.

# Help: List metric names

In [ ]:
!monasca help metric-name-list

# Example: List metric names

In [ ]:
!monasca metric-name-list

# Metrics Measurements
---
Operations for returning measurements.

* GET /v2.0/metrics/measurements

* Name and dimensions supplied as query parameters

* Startime, endtime, offset, limit and group_by supplied.

* Note:

    * If group_by is not specified, metrics must be fully qualified with name and dimensions.

# Help: List Measurements
---

In [ ]:
!monasca help measurement-list

# Excercise 2: List measurement
---
Get the list of 5 last measurements with the metric name 'monasca.thread_count' for service 'monitoring' within the last 5 minutes.

In [ ]:
!monasca measurement-list --limit 5 monasca.thread_count -- -5

# Exercise 3: List measurements by dimension
---
Get the list of measurements with the metric name 'fs.usage_bytes' within last 2 minutes. Please compare with the metric-list and try out using 'group_by' option.


In [ ]:
!monasca measurement-list fs.usage_bytes --group_by device -2

# Exercise 4: Create a function to get measurements
---
Let's try to use client's method 'metrics.list_measurements' to get the list of measurements.
Please use the output of the get_metrics() function defined in the previous example as the first argument of the new function. Remember that the metric is defined by its name and the unique set of dimensions (these are attributes of the metric).

In [ ]:
def get_measurements(metrics, start_time = None, end_time = None, limit=100):
    measurements = []
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:        
        # Invoke the Monasca client
        
        measurements += # put your code here
        
    return measurements

# TEST
metrics = get_metrics(['cpu.user_time','cpu.system_time'])
measurements = get_measurements(metrics, limit=2)
assert measurements[0]['name'] == 'cpu.user_time'
assert len(measurements[0]['measurements']) == 2
assert measurements[1]['name'] == 'cpu.system_time'

print json.dumps(measurements, indent=4)

# Notification Methods
---
Specify the name, type and address to send a notification to.

* GET, POST, PUT, DELETE, PATCH /v2.0/notification-methods

* Notification methods are associated with actions in alarms and are invoked when alarm state transitions occur.

* Supported notification methods are:

  * Email
  
  * PagerDuty
  
  * Webhooks
  
* Plugins available for:

  * Jira
  
  * HipChat
  
  * Slack
  
* Alarm grouping, silencing and inhibition [under development](https://review.openstack.org/434537)

# Delete all existing notifications
Let's first delete all existing notifications to prepare the service for creating new ones.

In [ ]:
notifications = monasca_client.notifications.list()

for notification in notifications:
    monasca_client.notifications.delete(
        notification_id = notification['id']
    )

# Help: Create Notification Method
---

In [ ]:
!monasca help notification-create

# Example: Create an email notification method

In [ ]:
!monasca notification-create "Email Notification" EMAIL john.doe@domain.com

# Example: Create HipChat notification method

In [ ]:
!monasca notification-type-list

In [ ]:
!monasca notification-create "HipChat Notification" HIPCHAT "https://monasca.hipchat.com/v2/room/3235013/notification?auth_token=8PtY96mOobVpGIuSucQZCJYLkRgC89OqECkRJprr"

Hipchat plugin is not enabled per default. Please add it to NF_PLUGINS environment variable in docker-compose.yml and restart the container.

In [ ]:
%pushd monasca-docker
!docker-compose up -d monasca-notification
%popd

# Help: List Notification Methods
---

In [ ]:
!monasca help notification-list

# Example: List Notification Methods

In [ ]:
!monasca notification-list -j

# Alarm Definitions
---
Operations for creating, reading updating and deleting alarm definitions.

* GET, POST /v2.0/alarm-definitions

* GET, PUT, PATCH, DELETE /v2.0/alarm-definitions/{alarm-definition-id}

* Alarm definitions are templates that are used to automatically create alarms based on matching metric names and dimensions

    * The `match-by` paramater is used to match/group metrics together by dimension
    
    * e.g. `--match-by hostname` will create an alarm per unique hostname.

* One alarm definition can result in many alarms

* Simple grammar for creating compound alarm expressions:

   avg(cpu.user_perc{}) > 85 or avg(disk.read_ops{device=vda}, 120) > 1000


* Alarm state (ALARM, OK and UNDETERMINED)

* Actions (notification methods) associated with alarms for state transitions

* User assigned severity (LOW, MEDIUM, HIGH, CRITICAL)



# Delete existing alarm definitions
Let's first delete all existing alarm definitions to prepare the service for creating new ones.

In [ ]:
alarm_definitions = monasca_client.alarm_definitions.list()

for definition in alarm_definitions:
    monasca_client.alarm_definitions.delete(alarm_id=definition['id'])    

# Help: Create alarm definition

In [ ]:
!monasca help alarm-definition-create

# Example: Create alarm definition
Create an alarm definition that triggers if the maximum of CPU user percent is greater than 80% over 3 periods of 60 seconds.

In [ ]:
!monasca alarm-definition-create simple-alarm "max(cpu.user_time_sec{}, 60) > 0.8 times 3"

# Example: Create a compound alarm definition
An alarm definition that triggers if either the CPU user or system percent is greater than 80% over 3 periods of 60 seconds.

In [ ]:
!monasca alarm-definition-create compound-alarm "avg(cpu.user_time_sec{}, 60) > 0.8 times 3 or avg(cpu.system_time_sec{}, 60) > 0.8 times 3"

# Alarm definition match_by

* As metrics are consumed by the Threshold Engine they are filtered and checked if they match the definitions that have been specified, based on the metric name and dimensions.

* If a match is found and it is the first one, a new alarm is created.

* If the match_by parameter is specified, metrics are grouped into alarms that match the specified dimensions.

# Example: Two alarm definitions
1. One with no match_by parameter
2. One with match_by set on hostname

In [ ]:
!monasca alarm-definition-create "match name only" "max(test-metric{}) > 80"
!monasca alarm-definition-create --match-by hostname "match by hostname" "max(test-metric{}) > 80"

# Example: Send two metrics
1. One with a name of test-metric and a hostname set to foo
2. One with a name of test-metric and a hostname set to bar

In [ ]:
!monasca metric-create --dimensions hostname=foo test-metric 0.0
!monasca metric-create --dimensions hostname=bar test-metric 0.0

# Quiz: How many alarms were created?

In [ ]:
!monasca alarm-list --metric-name "test-metric" -j

Three alarms were created:

1. There is one alarm with name "match name only". It has two metrics that are associated with it.

2. There are two alarms with name "match by hostname". Each alarm has one metric that is associated with it, based on the hostnames of `foo` and `bar`.

# UNDETERMINED alarm state
* There are three states of an alarm, `OK`, `ALARM` and `UNDETERMINED`.

* The `UNDETERMINED` alarm state occurs when metrics are no longer being received by an alarm.

* For example, a service is disabled, a system goes down, the network experiences a disruption.

* You can define the alarm as `deterministic`. The only allowed states are then: `OK` and `ALARM`.

# Example: Create an alarm definition with an alarm action.

In [ ]:
# Get the ID of the HipChat notification method
notifications = monasca_client.notifications.list(sort_by='type')
for notification in notifications:
    if notification['type'] == 'HIPCHAT':
        hipchat_notification = notification   
        hipchat_notification_id = hipchat_notification['id']

# Create the alarm
monasca_client.alarm_definitions.create(name='bootcamp-test',
                                        expression='max(cpu.user_time_sec) > 1',
                                        alarm_actions=[])

# Stress the system and wait for notification
---
* Open [HipChat room](https://www.hipchat.com/gxUiEZgyf)
* Stress the system
* Wait for your notification

In [ ]:
!stress-ng -c 2 -i 1 -m 1 --vm-bytes 128M -t 60s

# Alarms
---
Alarms are created when incoming metrics match alarm definitions


* GET /v2.0/alarms
* GET, PUT, PATCH, DELETE /v2.0/alarms/{alarm-id}

* Query Parameters

    * alarm_definition_id (string, optional) - Alarm definition ID to filter by.

    * metric_name (string(255), optional) - Name of metric to filter by.

    * metric_dimensions ({string(255): string(255)}, optional) - Dimensions of metrics to filter by specified as a comma separated array of (key, value) pairs as `key1:value1,key1:value1, ...`

    * state (string, optional) - State of alarm to filter by, either `OK`, `ALARM` or `UNDETERMINED`.

    * state_updated_start_time (string, optional) - The start time in ISO 8601 combined date and time format in UTC.
    
    * sort-by: Fields to sort by
    
    * offset, limit


# Help: Alarms list

In [ ]:
!monasca help alarm-list

# Example: Alarm list

In [ ]:
!monasca alarm-list --sort-by severity -j

# Alarm Counts
---
Provides the ability to query counts of alarms using a number of filter and group-by query parameters.

* GET /v2.0/alarms/counts

* Primarily used in summary/overview dashboards to show the number of alarms in the OK, ALARM and UNDETERMINED state.

* Queries are processed in-database, not client-side.

* A number of filter and group-by fields are supported.

# Help: Alarm counts

In [ ]:
!monasca help alarm-count

# Example: Alarm counts

In [ ]:
!monasca alarm-count --metric-dimensions service=monitoring --group-by state,dimension_name

# Metrics Agent
---
* A Python monitoring agent
* Push model
* Agent is installed on the systems that we want to monitor
* Collects metrics by running a set of collection plugins every X amount of seconds 
* Collection plugins are enabled by detection plugins
* Detection plugins generate yaml config files that the collection plugins read from
* The agent has a monasca-setup command line tool that helps configure the agent and run detection plugins

# Agent Metrics and Plugins

* System metrics (cpu, memory, network, filesystem, …)

* Service metrics

* RabbitMQ, MySQL, Kafka, and many others

* Application metrics

    * Built-in statsd daemon

    * Python monasca-statsd library: Adds support for dimensions
    
    * Prometheus metrics [in development](https://review.openstack.org/401413/)

* VM system metrics

* Kubernetes metrics

* Active checks

    * HTTP status checks and response times
    
    * System up/down checks (ping and ssh)
    
* Support for Nagios and checkmk

* Extensible/Pluggable: Additional services can be easily added


# Monasca-setup options

In [ ]:
!/opt/monasca-agent/bin/monasca-setup --help

In [ ]:
!sudo /opt/monasca-agent/bin/monasca-setup --username monasca-agent --password $PASSWORD --project_name $PROJECT_NAME \
               --keystone_url $KEYSTONE_URL --project_domain_name Default --user_domain_name Default \
               --system_only

# Monasca agent configuration file

In [ ]:
!sudo cat /etc/monasca/agent/agent.yaml

# Agent Detection Plugins

* Run after initial configuration is run
* [List of avaiable plugins](https://github.com/openstack/monasca-agent/blob/master/docs/Plugins.md)


# Run httpcheck Detection Plugin

In [ ]:
!sudo  /opt/monasca-agent/bin/monasca-setup -d httpcheck -a 'url=http://keystone:5000/'

# Example: Detection yaml configuration

In [ ]:
!sudo cat /etc/monasca/agent/conf.d/http_check.yaml

# Example: Query the Kafka consumer lag metrics

In [ ]:
!monasca metric-list --name http_response_time

# Example: Query the Kafka consumer lag measurements

In [ ]:
!monasca measurement-list http_response_time -10 --limit 10

# Logs Agent
---
* Based on Logstash (alternatively Beaver)

* Uses plugins

    * file for input
    
    * [monasca-log-api](https://github.com/logstash-plugins/logstash-output-monasca_log_api) for output

# Example: Examine the configuration of log agent

In [ ]:
!cat /opt/stack/monasca-log-agent/agent.conf

# Example: Collect logs of metric agent
---
* Edit the configuration of log-agent

    nano /opt/stack/monasca-log-agent/agent.conf
    
* Add the following block to the input plugin configuration

    file {
        add_field => { "dimensions" => { "service" => "monasca" }}
        add_field => { "dimensions" => { "component" => "monasca-agent" }}
        path => "/var/log/monasca/agent/*"
    }
    
* Save the new configuration    

# Test the syntax of new configuration

In [ ]:
!/opt/stack/logstash/bin/logstash -f /opt/stack/monasca-log-agent/agent.conf --configtest

# Restart the service

In [ ]:
!sudo systemctl restart monasca-log-agent

# Horizon, Grafana and Kibana Demo
---
* Log in to the Horizon dashbord 
    
    * URL: `http://<your_instance_ip>`
    
    * User: mini-mon
    
    * Password: password

# Monitoring Dashboard
---
* Explore the panels

    * Overview
    * Alarm Definitions
    * Alarms
    * Notifications

# Grafana 4
---
* Open Grafana from Monitoring Overview

![Open_Grafana](Open_Grafana.png)

# Add Monasca Datasource
---
* Select *Data Sources -> Add data source*
* Configuration:

    * Name: monasca
    * Type: Monasca
    * Default: yes
    * Url: `http://<your_instance_ip>:8070`
    * Auth: Keystone Auth
    
![Add Data Source](Add_DataSource.png)

# Kibana
---
* Open Log Management from Monitoring Overview
* Monasca Kibana plugin enhancements:
    * OpenStack user role based authorization
    * Set default index pattern based on the current project
    * Filter queries to authorized project only
* Explore the panels:
    * Discover
    * Visualize
    * Dashboard

# Contact the Monasca Team
---
Here are a couple of ways of getting in-touch with us.

* Monasca Weekly Meetings

    * Weekly on Wednesday at 1500 UTC in #openstack-meeting-3 (IRC webclient)
    
* Monasca in IRC

    * #openstack-monasca

# OpenStack Monitoring Questionnaire
---

Please help us to understand your needs by completing this [survey](https://goo.gl/forms/r1fFzMg9PNB4RiJC2).
It will take only 1-2 minutes.

# Thank you

# Solution ex. 1:

In [ ]:
!monasca metric-list --name openstack.handson_status --dimensions 'city=Sydney'

# Solution ex. 2:

In [ ]:
!monasca measurement-list --dimensions 'service=monitoring' --limit 5 monasca.thread_count -5   

# Solution ex. 3:

In [ ]:
!monasca metric-list --name fs.usage_bytes
!monasca measurement-list --group_by device fs.usage_bytes -2

# Solution ex. 4:

In [ ]:
measurements += monasca_client.metrics.list_measurements(
                name=metric['name'],
                dimensions=metric['dimensions'],
                start_time=start_time,
                end_time=end_time,
                limit=limit)